In [1]:
import tensorflow as tf
import math
import time
import numpy as np

In [2]:
from IPython.display import clear_output
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
!cp /content/drive/MyDrive/Arcface/backbone_0.py .
!cp /content/drive/MyDrive/Arcface/hypar.py .
!cp /content/drive/MyDrive/Arcface/network_16.py .
import hypar
import backbone_0 as nn
import network_16 as net

In [5]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
assert len(physical_devices) > 0, "Not enough GPU hardware devices available"
config = tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [6]:
physical_devices

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [7]:
X = np.load('/content/drive/MyDrive/Arcface/x_train.npy', allow_pickle=True)
Y = np.load('/content/drive/MyDrive/Arcface/y_train.npy', allow_pickle=True)

In [8]:
X = np.array(X, dtype='float32')
Y = np.array(Y, dtype='int32')
Y = np.reshape(Y, Y.shape[0])
print("X shape:",X.shape,"Y shape:",Y.shape) 
X = net.Resnet_preprocess(X)
images = X
labels = Y

# Prepare the training dataset.
train_dataset = tf.data.Dataset.from_tensor_slices((images, labels))
train_dataset = train_dataset.shuffle(buffer_size=1024).batch(hypar.batch_size)
print("Training dataset ready!")

X shape: (1476, 112, 112, 3) Y shape: (1476,)
Training dataset ready!


In [9]:
num_classes = hypar.num_classes #number of people in the dataset

#feature vector dimension = (512) [comes from the resnet model]

class Arcface_Layer(tf.keras.layers.Layer):                                     # Arcface layer definition
    def __init__(self, num_outputs = num_classes, s=64., m=0.5):                 # s is scale factor, m is the margin to be added to the angle 'theta'
        self.output_dim = num_outputs
        self.s = s
        self.m = m
        super(Arcface_Layer, self).__init__()

    def build(self, input_shape):

        self.kernel = self.add_weight(name='weight',
                                          shape=(input_shape[-1],self.output_dim),
                                          initializer='glorot_uniform',
                                          regularizer=tf.keras.regularizers.l2(l=5e-4),
                                          trainable=True)
        super(Arcface_Layer, self).build(input_shape)


    def call(self, embedding, labels):
        cos_m = math.cos(self.m)
        sin_m = math.sin(self.m)
        mm = sin_m * self.m  # issue 1
        threshold = math.cos(math.pi - self.m)
        # inputs and weights norm
        embedding_norm = tf.norm(embedding, axis=1, keepdims=True)
        embedding = embedding / embedding_norm
        weights_norm = tf.norm(self.kernel, axis=0, keepdims=True)
        weights = self.kernel / weights_norm
        # cos(theta+m)
        cos_t = tf.matmul(embedding, weights, name='cos_t')
        cos_t2 = tf.square(cos_t, name='cos_2')
        sin_t2 = tf.subtract(1., cos_t2, name='sin_2')
        sin_t = tf.sqrt(sin_t2, name='sin_t')
        cos_mt = self.s * tf.subtract(tf.multiply(cos_t, cos_m),
                                      tf.multiply(sin_t, sin_m), name='cos_mt')

        # this condition controls the theta+m should in range [0, pi]
        #      0<=theta+m<=pi
        #     -m<=theta<=pi-m
        cond_v = cos_t - threshold
        cond = tf.cast(tf.nn.relu(cond_v, name='if_else'), dtype=tf.bool)

        keep_val = self.s * (cos_t - mm)
        cos_mt_temp = tf.where(cond, cos_mt, keep_val)

        mask = tf.one_hot(labels, depth=self.output_dim, name='one_hot_mask')
        # mask = tf.squeeze(mask, 1)
        inv_mask = tf.subtract(1., mask)

        s_cos_t = tf.multiply(self.s, cos_t, name='scalar_cos_t')

        output = tf.add(tf.multiply(s_cos_t, inv_mask), tf.multiply(cos_mt_temp, mask), name='arcface_loss_output')

        return output

In [10]:
class train_model(tf.keras.Model):
    def __init__(self):
        super(train_model, self).__init__()
        #self.resnet = net.Resnet_nn()
        self.resnet = net.Resnet()
        self.arcface = Arcface_Layer()

    def call(self, x, y):
        x = self.resnet(x)
        return self.arcface(x, y)

# Instantiate a loss function.
def loss_fxn(logits,labels):
    loss_fn = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=labels))
    return loss_fn
    
# Instantiate an optimizer to train the model.
learning_rate = 0.0005
optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=False)
#optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,name='Adam')

model = train_model()

@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        logits = model(images,labels)
        pred = tf.nn.softmax(logits)
        #inf_loss = loss_fxn(pred,labels)
        inf_loss = loss_fxn(logits,labels)
        loss = inf_loss
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    train_loss = tf.reduce_mean(loss)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(pred, axis=1, output_type=tf.dtypes.int32), tf.cast(labels,dtype = tf.int32)), dtype=tf.float32))
    inference_loss = tf.reduce_mean(inf_loss)
    regularization_loss = 0
    return accuracy, train_loss, inference_loss, regularization_loss

94781440/94765736 [==============================] - 1s 0us/step


In [ ]:
epochs = 20
reg_coef = 1.0

file_name = '/content/drive/MyDrive/Arcface/checkpoint/try4_16/model_weights_'
file_number = 10
file_name = file_name + str(file_number*epochs)+ '_epochs'
model = tf.keras.models.load_model(file_name)

In [ ]:
print(file_name)

/content/drive/MyDrive/Arcface/checkpoint/try4_16/model_weights_200_epochs


In [ ]:
random_out = model(X[0:3,:,:,:],Y[0:3])
loss_log = []

epochs = 50
reg_coef = 1.0
file_number = 5

for save_wt in range (0, 5):
  for epoch in range(epochs):
      print("\nStart of epoch %d" % (epoch,))
      start_time = time.time()

      # Iterate over the batches of the dataset.
      for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
          accuracy, train_loss, inference_loss, regularization_loss = train_step(x_batch_train, y_batch_train)
          if step % 20 == 0:
            loss_log.append(train_loss)
            print("Training loss (for one batch) at step %d: %.4f"% (step, float(train_loss)))
      
  file_number += 1
  file_name = '/content/drive/MyDrive/Arcface/checkpoint/try5_SGD_16/model_weights_'
  file_name = file_name + str(file_number*epochs)+ '_epochs'
  model.save(file_name)


Start of epoch 0
Training loss (for one batch) at step 0: 19.3352
Training loss (for one batch) at step 20: 20.1776
Training loss (for one batch) at step 40: 19.2703
Training loss (for one batch) at step 60: 19.6231
Training loss (for one batch) at step 80: 20.3035

Start of epoch 1
Training loss (for one batch) at step 0: 18.2051
Training loss (for one batch) at step 20: 19.0149
Training loss (for one batch) at step 40: 19.2837
Training loss (for one batch) at step 60: 19.2879
Training loss (for one batch) at step 80: 20.3084

Start of epoch 2
Training loss (for one batch) at step 0: 19.0954
Training loss (for one batch) at step 20: 19.3294
Training loss (for one batch) at step 40: 19.1632
Training loss (for one batch) at step 60: 18.6725
Training loss (for one batch) at step 80: 19.5484

Start of epoch 3
Training loss (for one batch) at step 0: 19.6071
Training loss (for one batch) at step 20: 19.5735
Training loss (for one batch) at step 40: 20.1970
Training loss (for one batch) at

In [ ]:
loss_log_data = np.array(loss_log)
np.save('/content/drive/MyDrive/Arcface/checkpoint/try5_SGD_16/loss_log_1.npy', loss_log_data)

In [ ]:
test_model = model.resnet
result = test_model.predict(X)
result.shape

Exception ignored in: <function IteratorResourceDeleter.__del__ at 0x7f3305652ef0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/data/ops/iterator_ops.py", line 546, in __del__
    handle=self._handle, deleter=self._deleter)
  File "/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gen_dataset_ops.py", line 1264, in delete_iterator
    _ctx, "DeleteIterator", name, handle, deleter)
KeyboardInterrupt: 


(1476, 512)

In [ ]:
index = []
for i in range (0,1470):
    if Y[i] == 25:
        index.append(i)

In [ ]:
a = np.dot(result[877,:], np.array(result[1308,:]).T)
print(a)

In [ ]:
from sklearn.preprocessing import normalize
results = normalize(result, axis = 0)
results.shape

In [ ]:
model.arcface.get_weights()